In [ ]:
import os

import ase.io
import numpy as np
from ase import Atoms
from ase.build import fcc100
from ase.calculators.emt import EMT
from ase.constraints import FixAtoms
from ase.io import extxyz
from ase.io.trajectory import Trajectory
from ase.optimize import LBFGS

# This cell sets up and runs a structural relaxation
# of a Cu(100) surface. It uses ASE scripts to make the surface.
# The actual surfaces in OC20 were prepared slightly differently.

# Make the bare slab using an ASE helper script
slab = fcc100("Cu", size=(3, 3, 3))

# Tag all slab atoms below surface as 0, surface as 1, adsorbate as 2
tags = np.zeros(len(slab))
tags[18:27] = 1
slab.set_tags(tags)

# Fixed atoms are prevented from moving during a structure relaxation.
# We fix all slab atoms beneath the surface.
cons = FixAtoms(indices=[atom.index for atom in slab if (atom.tag == 0)])
slab.set_constraint(cons)
slab.center(vacuum=13.0, axis=2)
slab.set_pbc(True)

# Set the toy calculator (EMT) so ASE knows how to get energies/forces
# at each step
slab.calc = EMT()

os.makedirs("data", exist_ok=True)

# Define structure optimizer - LBFGS. Run for 100 steps,
# or if the max force on all atoms (fmax) is below 0 ev/A.
# fmax is typically set to 0.01-0.05 eV/A,
# for this demo however we run for the full 100 steps.

dyn = LBFGS(slab, trajectory="data/Cu100.traj")
dyn.run(fmax=0.03, steps=100)

traj = ase.io.read("data/Cu100.traj", ":")

# convert traj format to extxyz format (used by OC20 dataset)
columns = ["symbols", "positions", "move_mask", "tags"]
with open("data/Cu100.extxyz", "w") as f:
    extxyz.write_xyz(f, traj, columns=columns)